# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png" width="500">

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****hPcA
LANGSMITH_API_KEY=****a745
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [4]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [5]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [6]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

## Build a Dynamic Prompt
Utilize runtime context and middleware to generate a dynamic prompt.

In [7]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in `create_agent`.

In [8]:
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
my_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro", 
    max_tokens=None,
    temperature=0 # Lower temperature for stable, factual SQL generation
)#new

agent = create_agent(
    model=my_llm,
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [9]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

[{'type': 'text', 'text': 'I am sorry, but I cannot answer your question. I only have access to the following tables: `Album`, `Artist`, `Genre`, `Playlist`, `PlaylistTrack`, and `Track`. I do not have access to customer or invoice information.', 'extras': {'signature': 'CuMWAdHtim9vIefwnkfCLW40EQzhOT9amDPPnk8HmP9nkDjRUsIIGX1WSY71wGmeOMEwsx8Jvop5m/UJyfAGpXLeAH/4/HzhFDu6fSqNO+pfII7pnOOcCVAH+wP8S6vlTD0BdeQA8rdtk0TU8GKF8GVZ9Rr9NZkmpDrPENKaTkrlP34zGD4wYe1skA+hES22NCpPg+t9GJ4WqJD0fwjgS1Fl5yGo+OjiqlXf1J7TS79aTDRBqdEqb/Gtyto97pteulEa59Id88sR8uvsRggiQ5QngxXcaLY4oGd7lBdzl5vEyoDvpXYXBmk2Fwbrpm4L9tftorb/JYYGeVY0dm7PrCir1+PrS1fo3Zx4LnuX/pCtx91f7W+Jpxyv4gEqmNoF6zBHEXyCAaL+aIXggSVvVZjXrxA93Mreu+6EgRTHEOAf3Pw01RkKKZXdXYXgPLapQsYEZGSbM6Zmz2jLqguUsXJCKTcot3bTbMsumIh+C5TkWv4BcwoGXZv30cxUB8sBiauio85ESjaGQv/3C8RkG0bW//JgxwyYVEwKVfmvPS+6rogYUqK++zjXuXxPAldPpaz3b7vSR5m2A6kIFCm3Kf3Cxb3sPvCaHn/HfkTzr4+5yMuoKgAle+FQhl1aqKYIBQuyzn

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [10]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  execute_sql (878ac419-7efd-444c-865f-8c7133fc75c6)
 Call ID: 878ac419-7efd-444c-865f-8c7133fc75c6
  Args:
    query: SELECT
  T1.FirstName,
  T1.LastName,
  T2.InvoiceId,
  SUM(T3.UnitPrice * T3.Quantity) AS Total
FROM customers AS T1
INNER JOIN invoices AS T2
  ON T1.CustomerId = T2.CustomerId
INNER JOIN invoice_items AS T3
  ON T2.InvoiceId = T3.InvoiceId
WHERE
  T1.FirstName = 'Frank' AND T1.LastName = 'Harris'
GROUP BY
  T2.InvoiceId
ORDER BY
  Total DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT
  T1.FirstName,
  T1.LastName,
  T2.InvoiceId,
  SUM(T3.UnitPrice * T3.Quantity) AS Total
FROM customers AS T1
INNER JOIN invoices AS T2
  ON T1.CustomerId = T2.CustomerId
INNER JOIN invoice_items AS T3
  ON T2.InvoiceId = T3.InvoiceId
WHERE
  T1.FirstName = '

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

[{'type': 'text', 'text': "I made a mistake in the table names. I need to find the correct table names first. I'll start by listing all tables in the database.", 'extras': {'signature': 'CroDAdHtim8F7GKOMkHnXVxsyuhYbRkSPOkMnSHqd6iOzhQLx0RxLbHwmUZdPqLSG1up7hJJ2ubK2XvL0TVZ7+FezD9Kcco9708QusxF0yJquJZ1qEnPpfEfUM2gWHCXWqMIJM2Th+naffs+lu6FIY2VdnMpbdcCUUWejF/VsvhUdrmkSYNJOvtYOGjD6SdxHyeBm9trKENCWv6U7qI/4Ho0vshYR+NfI5VJk/3SyQ6gb4WDt2ba7fndWIETDmagKjVlRSE8jPdP6GjUE+04b8cVZSI5xVxxvaOmWGXflEEKyAILkNLUojNm8xsVLGDkafHzBxY50LPkI1kx+G3CD0/XfuvEQAA/HRBJUFOb6uVjlpjH5dwelH/vWeFY+CXC1/bAr2lwomTruNTmLkEx4Y3+oRdXzWD26Gi8r+Pkp7kV7JjPwwer1ZekCytZrpvOOf24+HE9OQROjDjatajmFToUeiA0grdPROSZqblWYsAFULMXVhhJwWV30hj0emSC7yEKp65INkv29pfTHLho5BuMUW7ZPkxei7O10NxEf9+/wNA/P3a1lgl/EVVSgGJrnlquNzWgSu8HjdiXVQ=='}}]
Tool Calls:
  execute_sql (87e980f2-7412-44cb-b99a-6e9106cd8d23)
 Call ID: 87e980f2-7412-44cb-b99a-6e9106cd8d23
  Args:
    query:

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2, model: gemini-2.5-pro
Please retry in 6.62623058s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelangua

================================== Ai Message ==================================
Tool Calls:
  execute_sql (77e36f2b-4173-4cd6-bd57-459b104c6d34)
 Call ID: 77e36f2b-4173-4cd6-bd57-459b104c6d34
  Args:
    query: SELECT
  T1.FirstName,
  T1.LastName,
  T2.InvoiceId,
  SUM(T3.UnitPrice * T3.Quantity) AS Total
FROM Customer AS T1
INNER JOIN Invoice AS T2
  ON T1.CustomerId = T2.CustomerId
INNER JOIN InvoiceLine AS T3
  ON T2.InvoiceId = T3.InvoiceId
WHERE
  T1.FirstName = 'Frank' AND T1.LastName = 'Harris'
GROUP BY
  T2.InvoiceId
ORDER BY
  Total DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[('Frank', 'Harris', 145, 13.86)]


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2, model: gemini-2.5-pro
Please retry in 4.578293304s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelangu

================================== Ai Message ==================================

[{'type': 'text', 'text': 'The most costly purchase by Frank Harris was for $13.86.', 'extras': {'signature': 'Co8NAdHtim+czjgBXKsFqF7KykAC0owj/shhkcGWqIS1O+fO6ZxKs+jcKVhOVIroykHTAuDTL8zTOynJrSHM9GJAu/3PVyB94s2InwsuClvS+Jzw56ODfyiNFq0R/tfS6oF04qzvXV+f68a75lvrIy7ny0Nl6e/JF0l3w+qFfeHDvnZFO/M/08AWBoE2Qrf4ol6iho9uZu7ICPmK3gM4wienyxTMduaqJqD2Wyy+aZMNEX/K33jpNuuUg/tUOIntu5KGQySbmMjZUMRfL2v8JfCYy/RSpLIGiOGutH0h/6BwqRZHwQwnJAL9ws1PyK6eygHAOv2HmWeoButnDjzY4FnE3/R5iQd5BVbJJJfQpkaGFcvK4kJAkFKyc2g9Y/IDEpBjp77mEOWsx5Glmj+hwJM5ICmXdBAkV73e/z/Z0gtZzWXjQIKudqx8L0YFeoEkvhpLgt/y/5MbHD5IobHKYKp0HBZ9CPbeAhqFMeYdhw5bTW7XHi0nJTmjTWJWo1voJ7tto+qM8ZhfKUMVMwVVPeUpTlHtQl+zPTzgB34ySf8dx31z75T11tm1B6bn2TnRGDo8xZf1dpcFlTW7xfU4/jIA+umDRJBgI6BvJn67TUwcnAE+kMARWR+9vY0VkLvXiJmxv0Z04dTLxaz7Fnu0y8AZT6KFZjTOGJnZEqbf/jqV6es9nR0bqp3VBRUfSbK3lyE/o2ND07VDIBCTog1JN6ckul3FeJyoG/XjO0QPl6c81URfg8QChHkD11dvFVtgptV6rDbY1wPzuLfDKE+Xt+i41OB76UhojcpOrsJ

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
